In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [2]:
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

## Загрузка и предобработка данных

In [3]:
!mkdir train_data test_data

mkdir: cannot create directory ‘train_data’: File exists
mkdir: cannot create directory ‘test_data’: File exists


In [13]:
!mkdir train_data/class_0 test_data/class_0

In [3]:
df = pd.read_feather('burst_data_all.fth')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,15298,15299,15300,15301,15302,15303,15304,15305,15306,15307
0,0.007503,0.010015,0.010986,0.009562,0.006206,0.011565,0.010189,0.014302,0.009023,0.011970,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.004064,0.004700,-0.001783,0.006729,0.006819,0.007735,0.005359,0.006582,0.005350,0.005619,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.015112,0.020660,0.014745,0.031673,0.029807,0.025416,0.021048,0.028070,0.022229,0.021666,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000142,0.000277,-0.000076,0.000288,-0.000010,0.000061,0.000127,0.000202,0.000193,-0.000091,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000431,0.000405,0.000806,0.000002,0.000299,0.000284,0.000187,0.000166,0.000466,0.000377,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
%cd train_data/class_0

for i in tqdm(df.index[:1100]):
  plt.plot(df.iloc[i,:1000])
  plt.axis('off')
  plt.savefig(f"gb{i}.png", bbox_inches='tight')
  plt.close()

%cd ../..

/content/train_data/class_0


100%|██████████| 1100/1100 [00:49<00:00, 22.01it/s]

/content


In [15]:
%cd test_data/class_0

for i in tqdm(df.index[1100:]):
  plt.plot(df.iloc[i,:1000])
  plt.axis('off')
  plt.savefig(f"gb{i}.png", bbox_inches='tight')
  plt.close()

%cd ../..

/content/test_data/class_0


100%|██████████| 249/249 [00:12<00:00, 19.24it/s]

/content


## Обучение модели 

In [4]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt # plotting library
import numpy as np # this module is useful to work with numerical arrays
import pandas as pd 
import random 
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
data_transform_ = transforms.Compose([
    transforms.Resize(size=(32, 32)),
    transforms.ToTensor()  
])

In [7]:
from torchvision import datasets

train_loader = datasets.ImageFolder(root='train_data', 
                                  transform=data_transform_  )

val_loader = datasets.ImageFolder(root='test_data', 
                                 transform=data_transform_)

print(f"Train data:\n{train_loader}\nTest data:\n{val_loader}")


Train data:
Dataset ImageFolder
    Number of datapoints: 1100
    Root location: train_data
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 249
    Root location: test_data
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )


In [18]:
!git clone https://github.com/axkoenig/autoencoder.git

Cloning into 'autoencoder'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 29 (delta 9), reused 24 (delta 8), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [24]:
!pip install -r autoencoder/requirements.txt

In [22]:
!python autoencoder/autoencoder.py --data_root train_data --gpus 1

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           3,072
       BatchNorm2d-2           [-1, 64, 64, 64]             128
         LeakyReLU-3           [-1, 64, 64, 64]               0
            Conv2d-4          [-1, 128, 32, 32]         131,072
       BatchNorm2d-5          [-1, 128, 32, 32]             256
         LeakyReLU-6          [-1, 128, 32, 32]               0
            Conv2d-7          [-1, 256, 16, 16]         524,288
       BatchNorm2d-8          [-1, 256, 16, 16]             512
         LeakyReLU-9          [-1, 256, 16, 16]               0
           Conv2d-10            [-1, 512, 8, 8]       2,097,152
      BatchNorm2d-11            [-1, 512, 8, 8]           1,024
        LeakyReLU-12            [-1, 512, 8, 8]               0
           Conv2d-13           [-1, 1024, 4, 4]       8,388,608
      BatchNorm2d-14           [-1, 102